# Simple notebook pipeline 

Welcome to your first steps with Kubeflow Pipelines (KFP). This notebook demos: 

* Defining a Kubeflow pipeline with the KFP SDK
* Creating an experiment and submitting pipelines to the KFP run time environment using the KFP SDK 

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/

### Prerequisites: Install or update the pipelines SDK

You may need to **restart your notebook kernel** after updating the KFP sdk.

This notebook is intended to be run from a Kubeflow notebook server. (From other environments, you would need to pass different arguments to the `kfp.Client` constructor.)

In [ ]:
# You may need to restart your notebook kernel after updating the kfp sdk
!python3 -m pip install kfp --upgrade --user

### Setup


In [1]:
EXPERIMENT_NAME = 'Simple notebook pipeline 6'        # Name of the experiment in the UI
BASE_IMAGE = 'tensorflow/tensorflow:2.0.0b0-py3'    # Base image used for components in the pipeline

In [2]:
import kfp
import kfp.dsl as dsl
from kfp import compiler
from kfp import components

### Create pipeline component

#### Create a python function

In [3]:
@dsl.python_component(
    name='add_op',
    description='adds two numbers',
    base_image=BASE_IMAGE  # you can define the base image here, or when you build in the next step. 
)
def add(a: float, b: float) -> float:
    '''Calculates sum of two arguments'''
    print(a, '+', b, '=', a + b)
    return a + b

/home/ajinkya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated function (or staticmethod) python_component. (This decorator does not seem to be used, so we deprecate it. If you need this decorator, please create an issue at https://github.com/kubeflow/pipelines/issues) -- Deprecated since version 0.2.6.
  after removing the cwd from sys.path.


#### Build a pipeline component from the function

In [4]:
# Convert the function to a pipeline operation.
add_op = components.func_to_container_op(
    add,
    base_image=BASE_IMAGE, 
)

### Build a pipeline using the component

In [5]:
@dsl.pipeline(
   name='Calculation pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def calc_pipeline(
   a: float =0,
   b: float =7
):
    #Passing pipeline parameter and a constant value as operation arguments
    add_task = add_op(a, 4) #Returns a dsl.ContainerOp class instance. 
    
    #You can create explicit dependency between the tasks using xyz_task.after(abc_task)
    add_2_task = add_op(a, b)
    
    add_3_task = add_op(add_task.output, add_2_task.output)

### Compile and run the pipeline

Kubeflow Pipelines lets you group pipeline runs by *Experiments*. You can create a new experiment, or call `kfp.Client().list_experiments()` to see existing ones.
If you don't specify the experiment name, the `Default` experiment will be used.

You can directly run a pipeline given its function definition:

In [10]:
# Specify pipeline argument values
arguments = {'a': '7', 'b': '8'}
# Launch a pipeline run given the pipeline function definition
kfp.Client(host='pipelines-api.kubeflow.svc.cluster.local:8888').create_run_from_pipeline_func(calc_pipeline, arguments=arguments, 
                                           experiment_name=EXPERIMENT_NAME)
# The generated links below lead to the Experiment page and the pipeline run details page, respectively

MaxRetryError: HTTPConnectionPool(host='pipelines-api.kubeflow.svc.cluster.local', port=8888): Max retries exceeded with url: /apis/v1beta1/experiments (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa579f18610>: Failed to establish a new connection: [Errno -2] Name or service not known'))

----
Copyright 2019 Google Inc. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.